In [ ]:
import torch
import csv
from tqdm import tqdm
from custom_dataset import EvalDataset
from custom_model import ConvTransformerAE, AEClassifier, CustomClassifier
from torch.utils.data import DataLoader
def eval_and_save_csv(
    model,
    dataloader,
    device,
    output_csv,
):
    model.eval()

    results = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            ids, x = batch
            x = x.to(device)

            logits, acc = model(x)              # (B, num_classes)
            preds = torch.argmax(logits, dim=1)

            ids = ids.cpu().numpy()
            preds = preds.cpu().numpy()

            for i in range(len(ids)):
                results.append((ids[i], int(preds[i])))

    # 写 CSV
    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["id", "pred_label"])
        writer.writerows(results)

    print(f"Saved results to {output_csv}")


In [ ]:
path = './resources/test_features_norm.npy'
dataset = EvalDataset(feature_path=path)
loader = DataLoader(
    dataset,
    batch_size=1024,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)
device = 'cuda'
# model
ae = ConvTransformerAE(
    input_dim=32,
    latent_dim=32,
    conv_channels=8,
    d_model=64,
    n_head=4,
    num_layers=10)
classifier = CustomClassifier(latent_dim=32, num_classes=5)
model = AEClassifier(ae, classifier)

model.load_state_dict(torch.load("./resources/models/classifier_model_35000.pth", map_location='cuda'))
model.to(device)

# eval
eval_and_save_csv(
    model=model,
    dataloader=loader,
    device=device,
    output_csv="predictions.csv"
)

In [ ]:
import numpy as np
num_classes = 5
labels = np.loadtxt('predictions.csv', delimiter=',', dtype=int, skiprows=1)
labels = labels[:, 1]
counts = np.bincount(labels.astype(np.int64), minlength=num_classes)
freq = counts / counts.sum()

print("Class counts:", counts)
print("Class freq:", freq)
